# Clasificación automatizada de cuentas con TFLearn

In [5]:
import nltk
from nltk.stem.lancaster import LancasterStemmer #permite transformar palabras para que sea mas entendible (letras de mas o errores de tipeo, ect).
stemmer = LancasterStemmer ()
import numpy
import tensorflow
import tflearn
import json
import random
import pickle #permite guardar el modelo y no cargarlo cada vez que se ejecute el programa
import csv
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


###         Carga de cuentas estándar desde archivo .json y balance a estandarizar

In [30]:
with open("estandar.json", encoding='utf-8') as archivo:
    datos = json.load(archivo)
    archivo.close()

#balanceFileName = 'balance_gam.csv'
# print(datos) #muestra de datos y prueba de carga correcta

{'cuentas': [{'id': '1.01.01.00', 'tag': 'Disponible', 'patrones': ['Disponible', 'CAJA PESOS', 'FONDO DE CAJA EN TIENDAS', 'FONDO FIJOS US$', 'FONDOS FIJOS EN MONEDA NACIONAL PESOS', 'CAJA LA SERENA', 'CAJA PARQUE ARAUCO', 'CAJA ANTOFAGASTA', 'CAJA MEGATIENDA', 'CAJA LA DEHESA', 'CAJA COPIAPO', 'CAJA TALCA', 'CAJA PLAZA TREBOL', 'CAJA CONCEPCION', 'CAJA TEMUCO', 'CAJA PUERTO MONTT', 'CAJA PLAZA VESPUCIO', 'CAJA VIÑA DEL MAR', 'CAJA RANCAGUA', 'CAJA SAC', 'CAJA COSTANERA CENTER', 'CAJA VALDIVIA', 'CAJA LOS DOMINICOS', 'CAJA ALTO LAS CONDES', 'CAJA OMNIUM', 'BANCO CRÉDITO E INVERSIONES PESOS', 'BANCO SANTANDER PESOS', 'BANCO SANTANDER PESOS  MEGACENTER', 'BANCO SANTANDER PESOS _ GARANTIAS', 'BANCO SCOTIABANK AZUL PESOS', 'BANCO INTER PESOS', 'BANCO SECURITY PESOS', 'BANCO CHILE PESOS', 'BANCO SCOTIABANK ROJO PESOS', 'BANCO BICE PESOS', 'BANCO ESTADO PESOS', 'BANCO ITAÚ PESOS', 'BANCO ITAU / CORP PESOS', 'BANCO ITAU LA SERENA', 'BANCO ITAU ARCADIA', 'BANCO ITAU PLAZA CROWNE AM', 'BANCO I

### Tokenizacion de cuentas y definición de parámetros

In [7]:
from tensorflow.python.framework import ops

# El 'try' permite almacenar los parametros del modelo para proximas ejecuciones. Se debe dejar comentado para las pruebas iniciales del sistema!!

########### Inicio comentado de almacenamiento ################################
# try: 
#     with open("variables.pickle","rb") as archivoPickle:
#         palabras, tags, entrenamiento, salida = pickle.load(archivoPickle)

# except:
###############################################################################

palabras=[]
tags=[]
auxX=[]  # palabras que se asignaran a la lista "palabras"
auxY=[]

for contenido in datos["cuentas"]:
    for patrones in contenido["patrones"]:
        auxPalabra = nltk.word_tokenize(patrones) # Toma una frase y la separa en palabras (tokens), para identificar los patrones a parametrizar
        palabras.extend(auxPalabra)               # Almacena todas las palabras tokenizadas 
        auxX.append(auxPalabra)                   
        auxY.append(contenido["tag"])

        if contenido["tag"] not in tags:          # Agrega un tag si no lo encuentra
            tags.append(contenido["tag"])

palabras = [stemmer.stem(w.lower()) for w in palabras if w != "\n"]   #stemmer permite identificar palabras equivalentes (por ejemplo: Banco = Bco. = banco)
palabras = sorted(list(set(palabras)))            # Se define la palabra
tags = sorted(tags)                               # Se define el tag

entrenamiento = []
salida = []
salidaVacia = [0 for _ in range(len(tags))]

for x, documento in enumerate(auxX):              # en X se guardara el indice de la palabra
    cubeta=[]                                     #Lista auxiliar para almacenar las palabras 
    auxPalabra = [stemmer.stem(w.lower()) for w in documento]
    for w in palabras:
        if w in auxPalabra:
            cubeta.append(1)
        else:
            cubeta.append(0)

    filaSalida = salidaVacia[:]
    filaSalida[tags.index(auxY[x])] = 1            #en la posicion del indice del tag, obtenemos el indice del auxiliar
    entrenamiento.append(cubeta)
    salida.append(filaSalida)

entrenamiento = numpy.array(entrenamiento)
salida = numpy.array(salida)

# ########### Inicio comentado de almacenamiento ##################################

#     with open("variables.pickle","wb") as archivoPickle:
#             pickle.dump((palabras, tags, entrenamiento, salida), archivoPickle)
            
# #################################################################################

### Entrenamiento de los datos
###### Se define el modelo a utilizar y los parámetros de entrenamiento

In [8]:

ops.reset_default_graph()


    ### Creación red neuronal inicial de 4 redes, con 10 neuronas c/u

red = tflearn.input_data(shape=[None,len(entrenamiento[0])]) # Datos de entrada a entrenar
red = tflearn.fully_connected(red,10)                        # Se crean capas conectadas con 10 unidades/neuronas
red = tflearn.fully_connected(red,10)
#red = tflearn.fully_connected(red,10)
red = tflearn.fully_connected(red,len(salida[0]),activation="softmax") # función de activación de las capas ocultas
red = tflearn.regression(red) #permite obtener la probabilidad de que tan eficaz es la clasificacion de los tag

modelo = tflearn.DNN(red)
#try:
#    modelo.load("modelo.tflearn")
#except:

historial = modelo.fit(entrenamiento,salida,n_epoch=40,batch_size=40,show_metric=True) #Entrenamiento para el proceso de modelado

modelo.save("modelo.tflearn")


Training Step: 2079  | total loss: 3.17335 | time: 0.167s
| Adam | epoch: 040 | loss: 3.17335 - acc: 0.2333 -- iter: 2040/2076
Training Step: 2080  | total loss: 3.16450 | time: 0.172s
| Adam | epoch: 040 | loss: 3.16450 - acc: 0.2400 -- iter: 2076/2076
--
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.data-00000-of-00001
INFO:tensorflow:200
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.index
INFO:tensorflow:200
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.meta
INFO:tensorflow:300


### Carga del balance a estandarizar (balance del cliente)

In [16]:
with open('balance_wesser.csv', newline='',encoding='utf-8-sig') as f:
    reader = csv.DictReader(f,delimiter=';')   
    cuentas = []
    for row in reader: 
        cuentas.append(row) 
        


### Definición del formato estandar

In [11]:
class Cuenta_cli:
    def __init__(self, id_estandar, id_cliente, nombre_cliente, activo_cliente, pasivo_cliente, perdida_cliente, ganancia_cliente):
        self.id_estandar = id_estandar
        self.id_cliente = id_cliente
        self.nombre_cliente = nombre_cliente
        self.activo_cliente = activo_cliente
        self.pasivo_cliente = pasivo_cliente
        self.perdida_cliente = perdida_cliente
        self.ganancia_cliente = ganancia_cliente


In [12]:
class Cuenta_est:
    def __init__(self, id_estandar, nombre_estandar, activo, pasivo, perdida, ganancia, listado_cliente):
        self.id_estandar = id_estandar
        self.nombre_estandar = nombre_estandar
        self.activo = activo
        self.pasivo = pasivo
        self.perdida = perdida
        self.ganancia = ganancia
        self.listado_cliente = []

In [32]:

def formatoEstandar(cuentas_clas, respuesta, data):
    lista_cuentas = []
    lista_estandar = []
    #print(cuentas_clas)
    #print(respuesta)
    #print(data)
    
    cont = 0
    for row in cuentas_clas:
        #lista_nombres_cliente.append(row)
        #lista_nombre_cliente = []
        #lista_id_cliente = []
        lista_clientes = []

        if(cont < len(respuesta)):
            #cliente = Cuenta_cli
            
            cliente = Cuenta_cli(row["id_cliente"], "x", row["nombre_cliente"], row["activo"], row["pasivo"], row["perdida"], row["ganancia"])
            
            #lista_nombre_cliente.append(row["\ufeffid_cliente"])
            #lista_id_cliente.append(row["nombre_cliente"])
            #cuenta = Cuenta_est
            #cuenta = Cuenta_est("", row["id_estandar"], "", respuesta[cont], row["activo"], row["pasivo"], row["perdida"], row["ganancia"])
            #cuenta.id_cliente = lista_id_cliente
            #cuenta.id_estandar = row["id_estandar"]
            #cuenta.nombre_cliente = lista_nombre_cliente
            #cuenta.nombre_estandar = respuesta[cont]
            #cuenta.activo = row["activo"]
            #cuenta.pasivo = row["pasivo"]
            #cuenta.perdida = row["perdida"]
            #cuenta.ganancia = row["ganancia"]
            #print(cuenta.nombre_cliente)
        cont = cont + 1
        lista_clientes.append(cuenta)

    #print(lista_cuentas[0].id_cliente)
    #print(lista_cuentas[0].nombre_cliente)
        ############################# Lista final para pasar a PDF ###############################
    
    for rec_data in data:
        esta = False
        sum_activos = 0
        sum_pasivos = 0
        sum_perdidas = 0
        sum_ganancias = 0
        for rec in lista_clientes:
            if(rec.id_estandar == rec_data["id"]):
                #print(rec.nombre_estandar)
                
                esta = True
                lista_id_cliente.append(rec.id_cliente[0])
                lista_nombre_cliente.append(rec.nombre_cliente[0])
                sum_activos = sum_activos + int(rec.activo)
                sum_pasivos = sum_pasivos + int(rec.pasivo)
                sum_perdidas = sum_perdidas + int(rec.perdida)
                sum_ganancias = sum_ganancias + int(rec.ganancia)
        
        if esta == True:
            #cuenta = Cuenta_est("", id_estandar, "", rec_data["tag"], sum_activos, sum_pasivos, sum_perdidas, sum_ganancias)
            cuenta.id_cliente = lista_id_cliente
            cuenta.nombre_cliente = lista_nombre_cliente
            cuenta.id_estandar = rec_data["id"]
            cuenta.activp = sum_activos
            cuenta.pasivo = sum_pasivos
            cuenta.perdida = sum_perdidas
            cuenta.ganancia = sum_ganancias
            
            lista_estandar.append(cuenta)
    
    #for rec in lista_estandar:
        #print(rec.nombre_cliente)
        #print(rec.nombre_estandar)
    return(lista_estandar)
    
    

### Creación PDF Balance final

In [14]:
from fpdf import FPDF

def agregarPDF(lista_cuentas):

    pdf = FPDF(orientation = 'L', unit = 'mm', format='A4') 
    pdf.add_page()

    pdf.add_font('DejaVu', '', 'DejaVuSansCondensed.ttf', uni=True)
    pdf.set_font('DejaVu', '', 9)

    # TEXTO
    #pdf.set_font('Arial', '', 10)

    # titulo
    pdf.cell(w = 0, h = 15, txt = 'Balance General', border = 1, ln=1, align = 'C', fill = 0)
    # encabezado

    pdf.cell(w = 30, h = 15, txt = 'ID Cuenta', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Activos', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Pasivos', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Perdida', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Ganancia', border = 1, align = 'C', fill = 0)
    pdf.multi_cell(w = 0, h = 15, txt = 'Descripcion', border = 1, align = 'C', fill = 0)


    #Tag: nombres de las cuentas resultantes de la estandarizacion
    #Datos
    #print(tag + "---------------- " + datos["nombre_cliente"])
    cont = 0
    for row in lista_cuentas:
        #print(respuesta[cont])           
        pdf.cell(w = 30, h = 9, txt = str(row.id_estandar), border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 9, txt = str(row.activo), border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 9, txt = str(row.pasivo), border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 9, txt = str(row.perdida), border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 9, txt = str(row.ganancia), border = 1, align = 'C', fill = 0)
        pdf.multi_cell(w = 0, h = 9, txt = row.nombre_estandar, border = 1, align = 'C', fill = 0)
        
    
    #pdf.output('hoja.pdf')
    pdf.output("hoja.pdf", 'F')
############################ Agregar elementos al PDF ##################################
        
        


### Interfaz para pruebas Sistema - Usuario

In [33]:

def mainBot():
    respuesta = []  # Temporal !!!
    
    #entrada = input("Usuario: ")
    count = 1   ############### Provisorio
    for x in range(len(cuentas) - 5):
        #print("-------------------",count,"-------------------")       ############### Provisorio
        aux = cuentas[x]
        indice = aux["nombre_cliente"]
        count = count + 1    ############### Provisorio
        cubeta = [0 for _ in range(len(palabras))]
        entradaProcesada = nltk.word_tokenize(indice)
        entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entradaProcesada]
        for palabraIndividual in entradaProcesada:
            for i,palabra in enumerate(palabras):
                if palabra == palabraIndividual:
                    cubeta[i] = 1
        resultados = modelo.predict([numpy.array(cubeta)])
        resultadosIndices = numpy.argmax(resultados)
        tag = tags[resultadosIndices]
        respuesta.append(tag)
        
        
        for tagAux in datos["cuentas"]:
            if tagAux["tag"] == tag:
                respuesta = tagAux["respuestas"]
                
        
        #print("\n\nUsuario: ",indice)
        #print("Sistema: ",random.choice(respuesta))
        
    
        
    lista_cuentas = formatoEstandar(cuentas, respuesta, datos["cuentas"])  
    
    #print(respuesta)
    
    agregarPDF(lista_cuentas)

    print("Programa ejecutado!")
mainBot()


{'cuentas': [{'id': '1.01.01.00', 'tag': 'Disponible', 'patrones': ['Disponible', 'CAJA PESOS', 'FONDO DE CAJA EN TIENDAS', 'FONDO FIJOS US$', 'FONDOS FIJOS EN MONEDA NACIONAL PESOS', 'CAJA LA SERENA', 'CAJA PARQUE ARAUCO', 'CAJA ANTOFAGASTA', 'CAJA MEGATIENDA', 'CAJA LA DEHESA', 'CAJA COPIAPO', 'CAJA TALCA', 'CAJA PLAZA TREBOL', 'CAJA CONCEPCION', 'CAJA TEMUCO', 'CAJA PUERTO MONTT', 'CAJA PLAZA VESPUCIO', 'CAJA VIÑA DEL MAR', 'CAJA RANCAGUA', 'CAJA SAC', 'CAJA COSTANERA CENTER', 'CAJA VALDIVIA', 'CAJA LOS DOMINICOS', 'CAJA ALTO LAS CONDES', 'CAJA OMNIUM', 'BANCO CRÉDITO E INVERSIONES PESOS', 'BANCO SANTANDER PESOS', 'BANCO SANTANDER PESOS  MEGACENTER', 'BANCO SANTANDER PESOS _ GARANTIAS', 'BANCO SCOTIABANK AZUL PESOS', 'BANCO INTER PESOS', 'BANCO SECURITY PESOS', 'BANCO CHILE PESOS', 'BANCO SCOTIABANK ROJO PESOS', 'BANCO BICE PESOS', 'BANCO ESTADO PESOS', 'BANCO ITAÚ PESOS', 'BANCO ITAU / CORP PESOS', 'BANCO ITAU LA SERENA', 'BANCO ITAU ARCADIA', 'BANCO ITAU PLAZA CROWNE AM', 'BANCO I

NameError: name 'cuenta' is not defined

In [3]:
import csv
with open('balance_wesser.csv', newline='',encoding='utf-8') as f:
    reader = csv.DictReader(f,delimiter=';')   
    cuentas = []
    for row in reader: 
        cuentas.append(row) 
        

#for row in cuentas:
#    print(row.nombre_estandar)

print(cuentas)

[{'\ufeffid_cliente': '11105', 'id_estandar': '1.01.01.00', 'nombre_estandar': 'Disponible', 'nombre_cliente': 'CAJA', 'activo': '33300000', 'pasivo': '33000000', 'perdida': '300000', 'ganancia': '0'}, {'\ufeffid_cliente': '11120', 'id_estandar': '1.01.01.00', 'nombre_estandar': 'Disponible', 'nombre_cliente': 'BANCO CREDITO E INVERSIONES', 'activo': '4038616765', 'pasivo': '3811012761', 'perdida': '227604004', 'ganancia': '0'}, {'\ufeffid_cliente': '11130', 'id_estandar': '1.01.01.00', 'nombre_estandar': 'Disponible', 'nombre_cliente': 'BCO.SANTANDER', 'activo': '432994736', 'pasivo': '423830093', 'perdida': '9164643', 'ganancia': '0'}, {'\ufeffid_cliente': '11140', 'id_estandar': '1.01.01.00', 'nombre_estandar': 'Disponible', 'nombre_cliente': 'BANCO DE CHILE', 'activo': '2198547836', 'pasivo': '2113800240', 'perdida': '84747596', 'ganancia': '0'}, {'\ufeffid_cliente': '11205', 'id_estandar': '1.01.03.00', 'nombre_estandar': 'Depósitos a plazo', 'nombre_cliente': 'DEPOSITOS A PLAZO',